In [1]:
import numpy as np
import numpy.random as rd
import sys
import matplotlib.pyplot as plt

dataset_num = 4
variance = 0.5
wtflag = 0
learning_rate = 0.3
file_exist = False

batch = 100
epochs = 100

# making dataset


def making_data_set():
    d_set = np.empty((1, 3), dtype=float)
    if file_exist:
        tmp = np.load(sys.argv[1])
        d_set = np.append(d_set, tmp)
    else:
        prob = rd.randint(0, 8, dataset_num)
        for i in prob:
            '''x = rd.normal(int(i/2)*2-1, variance)
            y = rd.normal(int(i/4)*2-1, variance)
            z = rd.normal(int(i/8)*2-1, variance)'''

            '''x = rd.normal(int(i / 2) * 2 - 1, variance)
            y = rd.normal(int(i / 4) * 2 - 1, variance)'''
            x = rd.randint(0, 2)
            y = rd.randint(0, 2)
            rst = x | y
            d_set = np.append(d_set, [[x, y, rst]], axis=0)
        if wtflag == 1:
            np.save('dataset.npy', d_set)
    d_set = np.delete(d_set, 0, 0)
    return d_set

t_data = making_data_set()
t_x_data = t_data[:, 0:-1]
t_y_data = t_data[:, -1].reshape(-1, 1)

test_data = making_data_set()
test_x_data = test_data[:, 0:-1]
test_y_data = test_data[:, -1].reshape(-1, 1)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class layer_class:
    w = np.array([]); bias = True;  dataflow = np.array([]);  delta = np.array([]);  y_ = 0
    def __init__(self, w, bias=False):
        self.w = w; self.bias = bias


class NN:
    train_set_x, train_set_y, test_set_x, test_set_y, learning_rate,  activate = np.array([]),np.array([]),np.array([]),np.array([]), 0, 'sigmoid'
    layer, train_loss, test_loss, accuracy = [], [], [], []
    feature = []
    def __init__(self, train_set_x, train_set_y, test_set_x, test_set_y, feature, learning_rate=0.01, activate='sigmoid'):
        self.learning_rate = learning_rate
        self.activate = activate
        self.train_set_x = train_set_x
        self.train_set_y = train_set_y
        self.test_set_x = test_set_x
        self.test_set_y = test_set_y
        self.feature = feature
        # w initialization sigmoid use xavier initialization  Relu using He initialization
        if activate is 'sigmoid':
            self.feature.insert(0, [self.train_set_x.shape[1], self.feature[0][1], self.feature[0][2]])
            for i, f in enumerate(self.feature):
                if i is (len(self.feature)-1):
                    break
                w = 0
                
                if f[2] is 'sigmoid':
                    w = rd.randn(f[0], int(self.feature[i + 1][0])) / np.sqrt(1/f[0])
                elif f[2] is 'relu':
                    w = rd.randn(f[0], int(self.feature[i + 1][0])) / np.sqrt(2/f[0])
                    
                if f[1] is True:
                    w = np.vstack(w, np.zeros((1, int(self.feature[i + 1][0]))))
                self.layer.append(layer_class(w))
            w = 0
            if self.feature[-1][2] is 'sigmoid':
                w = rd.randn(int(self.feature[-1][0]), self.train_set_y.shape[1]) / np.sqrt(1 / self.feature[-1][0])
            elif self.feature[-1][2] is 'relu':
                w = rd.randn(int(self.feature[-1][0]), self.train_set_y.shape[1]) / np.sqrt(2 / self.feature[-1][0])
            self.layer.append(layer_class(w))
            self.layer[0].dataflow = train_set_x


    def sigmoid(self, a):
        return a * (1 - a)
    def relu(self, a):
        a = a if a > 0 else 0
        return a
    def dsigmoid(self, a):
        return a * (1 - a)
    def drelu(self, a):
        if a > 0:
            return 1
        else:
            return 0

    # forwarding
    def pri(self):
        for i in self.layer:
            print(i.w)
        print("\n", self.layer[0].dataflow)

    def forwarding(self):
        ftn = 0
        if self.activate == 'sigmoid':
            ftn = self.sigmoid
        elif self.activate == 'relu':
            ftn = self.relu

        for i, k in enumerate(self.layer):
            a = np.array([])
            if i is 0:
                a = np.append(a, np.dot(self.train_set_x, k.w))
            else:
                a = np.append(a, np.dot(self.layer[i-1].dataflow, k.w))
            a = a.reshape(self.train_set_x.shape[0], k.w.shape[1])
            k.dataflow = ftn(a)
        self.train_loss = np.append(self.train_loss, np.average((self.layer[-1].dataflow - self.train_set_y) ** 2))

        for i, k in enumerate(self.layer):
            a = np.array([])
            if i is 0:
                a = np.append(a, np.dot(self.test_set_x, k.w))
            else:
                a = np.append(a, np.dot(self.layer[i-1].dataflow, k.w))
            a = a.reshape(self.test_set_x.shape[0], k.w.shape[1])
            k.dataflow = ftn(a)
        self.test_loss = np.append(self.test_loss, np.average((self.layer[-1].dataflow - self.train_set_y) ** 2))


    def backpropagation(self):
        dftn = 0
        if self.activate == 'sigmoid':
            dftn = self.dsigmoid
        elif self.activate == 'relu':
            dftn = self.drelu


        for i, L in reversed(list(enumerate(self.layer))):
            '''summation {}{j} wij delatj 교재 slide 12p'''

            # fully connected layer
            if i is (len(self.layer)-1):
                L.delta = np.multiply((self.train_set_y - L.dataflow), dftn(L.dataflow))
                #print(f"$$$$$$$$$i :{i}, L.dataflow : {L.dataflow.shape} L.delta : {L.delta.shape}, L.w : {L.w.shape}")
            #else
            else:
                back_L = self.layer[i+1]
                L.delta = np.multiply(dftn(L.dataflow), np.dot(back_L.delta, back_L.w.T))
                # print(f"L.delta 구하는 과정\n\ni :{i}, L.dataflow : {L.dataflow.shape} back_L.delta : {back_L.delta.shape}, back_L.w : {back_L.w.shape}, L.w. : {L.w.shape}\n 구하기 end \n\n")

            # i가 0 이면 input data가 train_set
            if i is 0 :
                input = self.train_set_x
            else:
                input = self.layer[i - 1].dataflow

            '''x_k * delta -> upgrade할 weight '''
            upgrade_w = np.zeros(1)
            #outer로 한꺼번에 추가후에, reshape로 정렬
            for k in range(L.delta.shape[0]):
                upgrade_w = np.append(upgrade_w, np.outer(input[k], L.delta[k]))
                #print(f" for {i} layer **********************\ndelta : \n{L.delta[k]}\n input : \n{input[k]}\n result : \n{np.outer(input[k], L.delta[k])}")
            upgrade_w = np.delete(upgrade_w, 0, axis=0)
            upgrade_w = upgrade_w.reshape(-1, L.w.shape[0], L.w.shape[1])

            """update weight"""
            L.w = L.w - np.average(upgrade_w, axis=0) * self.learning_rate

    def training(self, epochs):
        for i in range(epochs):
            self.forwarding()
            self.backpropagation()

            cnt = np.array(np.abs(test_y_data - self.layer[-1].dataflow) < 0.5)
            correct = np.average(cnt) * 100
            self.accuracy.append(correct)
            print(f"epochs : {i}\t train_loss :{self.train_loss[i]:.5}\t test_loss :{self.test_loss[i]:.5}, accuracy :{correct}")

    def plot(self):

        plt.figure()
        plt.plot(self.train_loss, 'b')
        plt.plot(self.test_loss, 'y')
        plt.show()

        plt.figure()
        plt.plot(self.accuracy)
        plt.show()




test = NN(t_x_data, t_y_data, test_x_data, test_y_data, [3, 5, 6], learning_rate=learning_rate)

test.training(epochs)
test.plot()



TypeError: 'int' object is not subscriptable